# Projet PHOTOS

# Fonction de copie des fichiers présents en base "catalogue photos"

## Reste à faire 

In [ ]:
#---------#
# TEST    #
#---------#

Test des copies de fichiers placés sur répertoire partagé situé sur le réseau.
=> vérification de l''impact de la latence.


In [ ]:
#----------------------------------------------------------------------------
#Récupération de la liste des répertoires en base (requête -> fichier)
#----------------------------------------------------------------------------
    #--------------------------------------------------
    #version actuelle : pas effectué
    #--------------------------------------------------

#----------------------------------------------------------------------------
#Lecture de la liste des répertoires en base (fichier) et affichage
#----------------------------------------------------------------------------
    #--------------------------------------------------
    #version actuelle : pas effectué
    #--------------------------------------------------

#----------------------------------------------------------------------------
#Choix du répertoire à copier
#----------------------------------------------------------------------------
    #--------------------------------------------------
    #version actuelle : pas effectué
    #--------------------------------------------------

#----------------------------------------------------------------------------
#Statistiques : nombre de sous-répertoires dans le répertoire
#----------------------------------------------------------------------------
    #--------------------------------------------------
    #version actuelle : pas calculé
    #--------------------------------------------------

#----------------------------------------------------------------------------------------------------
#Liste des sous-répertoires dans le répertoire (pour mise à jour de la liste des répertoires copiés)
#----------------------------------------------------------------------------------------------------
    #--------------------------------------------------
    #version actuelle : pas calculé
    #--------------------------------------------------

#----------------------------------------------------------------------------
#Statistiques : nombre de fichiers à copier
#----------------------------------------------------------------------------
    #--------------------------------------------------
    #version actuelle : pas calculé
    #--------------------------------------------------

#----------------------------------------------------------------------------
#Mise à jour de la liste des répertoires copiés
#----------------------------------------------------------------------------
    #--------------------------------------------------
    #version actuelle : pas calculé
    #--------------------------------------------------

In [2]:
#imports

import os
from os import listdir
from os.path import isfile, join, isdir

from PIL import Image, IptcImagePlugin
import numpy as np
from matplotlib import pyplot as plt # pour imgshow

import pandas as pd

import datetime as dt #pour timedelta
from datetime import datetime

import tqdm

import pymongo
from pymongo import MongoClient
#pour les images
import PIL.Image
import gridfs
#pour les images webp
#!pip3 install webptools  #webp... et pas web...
from webptools import dwebp
#pip install opencv-python (pour cv2)
#pour les vidéos
import cv2
import subprocess

import sys


## Création des fonctions de lecture du catalogue et d'insertion dans la base

In [3]:
def fnph_copy_file(parm_filename, parm_sourcename, parm_destname):
    
    import os
    import shutil

    #print("debug-->", "parm_filename",  parm_filename)
    #print("debug-->", "parm_sourcename",parm_sourcename)
    #print("debug-->", "parm_destname",  parm_destname)
    """
    Fonction de copie de fichier
    retourne un code status et un commentaire
    """
    myfile_source_fullname      = os.path.join(parm_sourcename, parm_filename)
    myfile_destination_fullname = os.path.join(parm_destname,   parm_filename)

    #print("debug-->", "myfile_source_fullname",      myfile_source_fullname)
    #print("debug-->", "myfile_destination_fullname", myfile_destination_fullname)

    
            #------------------#    
            # test de présence #
            #------------------#    
    
    if os.path.exists(myfile_destination_fullname): #ajout du test car shutil semble ne pas recopier (dcreation inchangée) 
                                                    #mais ne renvoie pas d'erreur (^donc n'écrase pas)
        status   = "ko-doublon"
        comment  = "Ce nom de fichier >" + parm_filename + "< existe déjà dans le répertoire de copie" 
        print("Debug-->doublon--> parm_filename=", parm_filename)
    else:
    
        try: 
            #------------------#    
            # copie du fichier #
            #------------------#    
            myfile_destination_created = shutil.copy2(myfile_source_fullname, myfile_destination_fullname) 
            status="ok"
            comment="File copied successfully."
            #print("Debug-->File copied successfully.") 
            #print('Debug-->done - file', myfile_destination_created, 'created.')


        # If source and destination are same 
        except shutil.SameFileError: 
            status="ko-SameFileError"
            comment="Source and destination represents the same file."
            print("\nSource and destination represents the same file.")
            print("------")
            print("myfile_source_name:")
            print(myfile_source_name)
            print("myfile_destination_fullname:")
            print(myfile_destination_fullname)
            print("------")

        # If there is any permission issue 
        except PermissionError: 
            status="ko-PermissionError"
            comment="Impossible to write. Permission denied."
            print("Permission denied.") 

        # For other errors 
        except: 
            status="ko-OtherError"
            comment="Error occurred while copying file."
            print("Error occurred while copying file.") 
            #print("Unexpected error:", sys.exc_info()[0])  ???
            #raise ???

    return[status, comment, myfile_source_fullname, myfile_destination_fullname]


In [4]:
#test def fnph_copy_file
import shutil
import os
parm_fname="Capturefondbl.JPG"
parm_sourcename=r"C:\Users\LENOVO\Documents\Projets\Prj_photos\Prjph_repertoire_test\Test_fonction_copie_source"
parm_destname  =r"C:\Users\LENOVO\Documents\Projets\Prj_photos\Prjph_repertoire_test\Test_fonction_copie_destination"

#parm_fname="Captureph1.PNG"
#parm_sourcename=r"C:\Users\LENOVO\Documents\Projets\Prj_photos\Prjph_repertoire_test\test_shutil"
#parm_destname  =r"C:\Users\LENOVO\Documents\Projets\Prj_photos\Prjph_repertoire_test\test_shutil\copies"

print(fnph_copy_file(parm_fname, parm_sourcename, parm_destname))
print("done")

Debug-->doublon--> parm_filename= Capturefondbl.JPG
['ko-doublon', 'Ce nom de fichier >Capturefondbl.JPG< existe déjà dans le répertoire de copie', 'C:\\Users\\LENOVO\\Documents\\Projets\\Prj_photos\\Prjph_repertoire_test\\Test_fonction_copie_source\\Capturefondbl.JPG', 'C:\\Users\\LENOVO\\Documents\\Projets\\Prj_photos\\Prjph_repertoire_test\\Test_fonction_copie_destination\\Capturefondbl.JPG']
done


## Programme principal

In [32]:
%%time
#Programme principal

#----------------
#initialisations
#----------------

#------ Répertoire de sauvegarde de l'historique des répertoires sauvegardés
parm_df_log_rep = r'C:\Users\LENOVO\Documents\Projets\Prj_photos\Prjph_log'

#Paramètres de connexion
#------ Base  -------------------*
client = pymongo.MongoClient('localhost',27017)
mydb = client["prjph_catalogue"]

#début Paramètres utilisateur *************************************************************************************

#------ Collection -------------------*      #1<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
mycollection_name = "test_documents"        #TEST
#mycollection_name = "images_videos"          #PREPROD
#------ ---------- -------------------*
mycollection = mydb[mycollection_name]

#------ Répertoire initial à copier ------*  #2<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
parm_source_dir_name= r"C:\Users\LENOVO\Documents\Projets\Prj_photos\Prjph_repertoire_test\Test_fonction_copie_source" 

#------ Répertoire destinataire des fichiers copiés -----*
parm_dest_dir_name  = r"C:\Users\LENOVO\Documents\Projets\Prj_photos\Prjph_repertoire_test\Test_fonction_copie_destination" #0b<<<<

#------ Libellé du traitement ------------*            #3<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
montitre_traitement  = "Copie de fichiers - "          #Est systématiquement complété plus bas par le nom du rep à copier
montitre_traitement += "(" + parm_source_dir_name +")"
#fin Paramètres****************************************************************************************************

#--------------------------------
#Tests de contrôle des fichiers
#--------------------------------

#test l'existence du répertoire de sauvegarde du log avant de démarrer
if not os.path.exists(parm_df_log_rep):
    print('ERREUR : le répertoire de sauvegarde n''existe pas')
    print('ERREUR : ' + parm_df_log_rep)
    print('ERREUR : veuillez vérifier et relancer')
    sys.exit("haa! errors! vérifier le nom du répertoire du log en paramètre et relancer")

#test l'existence du répertoire de destination des copies
if not os.path.exists(parm_dest_dir_name):
    print('ERREUR : le répertoire de copie n''existe pas')
    print('ERREUR : ' + parm_dest_dir_name)
    print('ERREUR : veuillez vérifier et relancer')
    sys.exit("haa! errors! vérifier le nom du répertoire de destination des copies et relancer")

#--------------------------------
#df historique des répertoires traités
#--------------------------------


#--------------------------------
#Compteurs - indicateurs
#--------------------------------
#nombre de fichiers copiés
cpt_nb_f_copy_ok     = 0
#nombre de fichiers à copier mais non copiés
cpt_nb_f_copy_failed = 0
#nombre de fichiers à copier mais déjà copiés (topés copiés dans la base)
cpt_nb_file_already_copied = 0

#global_unique_id : Variable de valeur unique pour rsgner unique_id lors des différents appels
global_unique_id_time_format = datetime.now()
global_unique_id             = str(datetime.timestamp(global_unique_id_time_format))

#-----------------------------------
# Displays de début de traitement
#-----------------------------------
start_time = datetime.now()
print('------------------------------------------------------------')
print('start......................... : ', start_time,         )
print('global_unique_id.............. : ', global_unique_id   )
print('collection.................... : ', mycollection_name   )
print('répertoire destination........ : ', parm_dest_dir_name  )
#----------------------------------------------------------------------------
#Connection au serveur mongodb 27017
#----------------------------------------------------------------------------
client = pymongo.MongoClient('localhost',27017)
mydb = client["prjph_catalogue"]

#----------------------------------------------------------------------------
#Récupération de la liste des répertoires en base (requête -> fichier)
#----------------------------------------------------------------------------
    #--------------------------------------------------
    #version actuelle : pas effectué
    #--------------------------------------------------

#----------------------------------------------------------------------------
#Lecture de la liste des répertoires en base (fichier) et affichage
#----------------------------------------------------------------------------
    #--------------------------------------------------
    #version actuelle : pas effectué
    #--------------------------------------------------

#----------------------------------------------------------------------------
#Choix du répertoire à copier
#----------------------------------------------------------------------------
    #--------------------------------------------------
    #version actuelle : pas effectué
    #--------------------------------------------------

print('directory asked to be copied.. : ', parm_source_dir_name,      '\n')

#----------------------------------------------------------------------------
#Détermination du critère de recherche des fichiers à copier (requête -> cursor)
#----------------------------------------------------------------------------
#le caractère spécial \ dans le nom de répertoire doit être doublé \\ pour être considéré comme \ non spécial
myquest = parm_source_dir_name.replace("\\", "\\\\")  #nb : dans cette écriture, on double également le car. spécial
                                                      #     pour dire que \ doit être remplacé par \\

#puis ajout de "^" pour spécifier que le nom doit commencer par... et pour obtenir tous les sous-répertoires
myquest = "^" + myquest
print('critère de recherche (myquest) : ', myquest)

#----------------------------------------------------------------------------
#Création du curseur des fichiers à copier (requête -> cursor)
#----------------------------------------------------------------------------
#cursor des fichiers à sauvegarder
mycursor_files = mycollection.find({"orgnl_dirname":{"$regex":myquest}})

#----------------------------------------------------------------------------
#Statistiques : nombre de fichiers dans le répertoire
#----------------------------------------------------------------------------
nb_files_in_rep = mycursor_files.count()
print('nb of files present in dir.... : ', nb_files_in_rep,           '\n')

#----------------------------------------------------------------------------
#Statistiques : nombre de sous-répertoires dans le répertoire
#----------------------------------------------------------------------------
    #--------------------------------------------------
    #version actuelle : pas calculé
    #--------------------------------------------------

#----------------------------------------------------------------------------------------------------
#Liste des sous-répertoires dans le répertoire (pour mise à jour de la liste des répertoires copiés)
#----------------------------------------------------------------------------------------------------
    #--------------------------------------------------
    #version actuelle : pas calculé
    #--------------------------------------------------



#----------------------------------------------------------------------------
#Statistiques : nombre de fichiers à copier
#----------------------------------------------------------------------------
    #--------------------------------------------------
    #version actuelle : pas calculé
    #--------------------------------------------------

#----------------------------------------------------------------------------
#Boucle sur les fichiers à copier
for f in mycursor_files:

#---Vérification du top copie (topé copié en base)
    if f["copy_events_fn"]["copy_e_last_status"]=="ok":
        flag_f_already_copied=True
    else:
        flag_f_already_copied=False
    
    #--------------------------------------------------
    #version actuelle : tous les fichiers sont copiés
    #--------------------------------------------------
    
#---Cas déjà copié (topé copié - status=ok - en base)
    if flag_f_already_copied:
        cpt_nb_file_already_copied += 1
        status = "ok"
        comment= "already copied - see database histo"
        print("debug-->" + f["filename"] + "< already copied - see database")
#---Cas pas copié  : lecture / copie
    else:
#------ copie via la fonction
        result = fnph_copy_file(f["filename"], f["orgnl_dirname"][0], parm_dest_dir_name) #[status, comment, source, dest]
        status = result[0]
        comment= result[1]
        source_full_name = result[2]
        dest_full_name   = result[3]
        
        if status == "ok":
            cpt_nb_f_copy_ok += 1
      
            
                
        else:
            cpt_nb_f_copy_failed += 1
                
    #------------------------------------------------------------#
    # Mise à jour du document (bd) copié (ou pas)  
    #------------------------------------------------------------#
    #set pour ajouter/màj un champ de type objet, push pour ajouter un élément dans un item de type liste"
    strdnow=str(datetime.now())
    my_copy_e_histo = [{
                "copy_eh_status" : status,
                "copy_eh_id"     : global_unique_id.replace('.','_'),
                "copy_eh_date"   : strdnow,
                "copy_eh_comment": comment,
                "copy_eh_source" : source_full_name,
                "copy_eh_dest"   : dest_full_name
                        }]       

    result = mycollection.update_one( 
                   {"_id":f["_id"]}, 
                   {'$set' :{
                                "copy_events_fn.copy_e_last_status" : status,
                                "copy_events_fn.copy_e_last_id"     : global_unique_id.replace('.','_'),
                                "copy_events_fn.copy_e_last_date"   : strdnow,
                                "copy_events_fn.copy_e_last_comment": comment,
                                "copy_events_fn.copy_e_last_source" : source_full_name,
                                "copy_events_fn.copy_e_last_dest"   : dest_full_name      
                            },
                    '$push':{   "copy_events_fn_histo":{"$each":my_copy_e_histo}}
                    }
                                     )

            
            
#----------------------------------------------------------------------------
#Mise à jour de la liste des répertoires copiés
#----------------------------------------------------------------------------
    #--------------------------------------------------
    #version actuelle : pas calculé
    #--------------------------------------------------

            
            
#-----------------------------------
# Displays de fin de traitement
#-----------------------------------
end_time = datetime.now()
print('ended......................... : ', end_time)
print('nb de fichiers à copier....... : ', 'non calculé')
print('nb de fichiers déjà copiés.... : ', cpt_nb_file_already_copied)
print('nb de fichiers copiés......... : ', cpt_nb_f_copy_ok)
print('nb de fichiers copie ko....... : ', cpt_nb_f_copy_failed)
print('------------------------------------------------------------')




#Programme principal fin



------------------------------------------------------------
start......................... :  2020-11-09 10:07:02.645213
global_unique_id.............. :  1604912822.645213
collection.................... :  test_documents
répertoire destination........ :  C:\Users\LENOVO\Documents\Projets\Prj_photos\Prjph_repertoire_test\Test_fonction_copie_destination
directory asked to be copied.. :  C:\Users\LENOVO\Documents\Projets\Prj_photos\Prjph_repertoire_test\Test_fonction_copie_source 

critère de recherche (myquest) :  ^C:\\Users\\LENOVO\\Documents\\Projets\\Prj_photos\\Prjph_repertoire_test\\Test_fonction_copie_source
nb of files present in dir.... :  40 

Debug-->doublon--> parm_filename= Capture d’écran (1).png
Debug-->doublon--> parm_filename= Capture d’écran (2).png
Debug-->doublon--> parm_filename= Capture d’écran (3).png
Debug-->doublon--> parm_filename= Capture d’écran (4).png
Debug-->doublon--> parm_filename= Capture d’écran (5).png
Debug-->doublon--> parm_filename= Capture.PNG
Deb

<timed exec>:129: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.


In [20]:
dnow=datetime.now()
str(dnow)
#strdnow=str(datetime.timestamp(dnow)).replace('.','_')
#strdnow

'2020-11-09 09:42:03.441998'